In [ ]:
# bismillahirrahmanirrahim

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install -q transformers datasets

In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from datasets import Dataset, DatasetDict
from PIL import Image
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from datasets import ClassLabel, Value, Sequence, Features
import pickle
from tqdm.auto import tqdm

In [ ]:
df = pd.read_csv('./drive/MyDrive/ITMO Master/datasets/ml-industry/task2_1/processed-multilabel.csv')
df.head(2)

,title,simple_desc,genre,img_local_path,labels,Action,Adventure,Animation,Biography,Comedy,...,Musical,Mystery,Romance,Sci-Fi,Short Film,Sport,Superhero,Thriller,War,Western
0,Andor,Prequel series to Star Wars' 'Rogue One'. In a...,Action,/content/drive/MyDrive/ITMO Master/datasets/ml...,"Action, Adventure, Drama",1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,House of the Dragon,An internal succession war within House Targar...,Action,/content/drive/MyDrive/ITMO Master/datasets/ml...,"Action, Adventure, Drama",1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
labels = list(df.columns)[5:]
label2id = {v: k for k, v in enumerate(labels)}
id2label = {k: v for k, v in enumerate(labels)}

len(labels)

24

In [ ]:
datasets = Dataset.from_pandas(df)
samples = datasets.select(range(6))

for sample in samples:
    label = ', '.join([k for k, v in list(sample.items())[5:] if v == 1])    
    print(sample['title'])
    print(sample['simple_desc'])
    print(f"Label: {label}")
    print('===============')    

Andor
Prequel series to Star Wars' 'Rogue One'. In an era filled with danger, deception and intrigue, Cassian will embark on the path that is destined to turn him into a Rebel hero.
Label: Action, Adventure, Drama
House of the Dragon
An internal succession war within House Targaryen at the height of its power, 172 years before the birth of Daenerys Targaryen.
Label: Action, Adventure, Drama
Warrior Nun
After waking up in a morgue, an orphaned teen discovers she now possesses superpowers as the chosen Halo Bearer for a secret sect of demon-hunting nuns.
Label: Action, Drama, Fantasy
Game of Thrones
Nine noble families fight for control over the lands of Westeros, while an ancient enemy returns after being dormant for millennia.
Label: Action, Adventure, Drama
The Lord of the Rings: The Rings of Power
Epic drama set thousands of years before the events of J.R.R. Tolkien's 'The Hobbit' and 'The Lord of the Rings' follows an ensemble cast of characters, both familiar and new, as they confr

In [ ]:
def get_label(example):
    example['labels'] = list(example.values())[5:]
    return example

In [ ]:
new_features = datasets.features.copy()
new_features['genre'] = ClassLabel(names=labels)
datasets = datasets.cast(new_features)

Casting the dataset:   0%|          | 0/7 [00:00<?, ?ba/s]

In [ ]:
# preprocess
dataset = datasets.map(get_label, remove_columns=datasets.column_names[5:])
dataset = dataset.train_test_split(0.2, seed=42, stratify_by_column='genre')
# make validation split
# split_dataset = dataset['train'].train_test_split(0.25, seed=42, stratify_by_column='genre')
# dataset['train'] = split_dataset['train']
# dataset['validation'] = split_dataset['test']

dataset

  0%|          | 0/6399 [00:00<?, ?ex/s]

DatasetDict({
    train: Dataset({
        features: ['title', 'simple_desc', 'genre', 'img_local_path', 'labels'],
        num_rows: 5119
    })
    test: Dataset({
        features: ['title', 'simple_desc', 'genre', 'img_local_path', 'labels'],
        num_rows: 1280
    })
})

In [ ]:
bart_ckpt = 'facebook/bart-base'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(bart_ckpt)

### Desc Model

In [ ]:
def desc_batch_transform(samples):
    # print(samples)
    # tokenize
    inputs = tokenizer([i for i in samples['simple_desc']],
                       max_length=128,
                       padding='max_length',
                       truncation=True,
                       return_attention_mask=True,
                       return_tensors='pt')
    # add labels
    # inputs['labels'] = [torch.Tensor(i).float() for i in samples['labels']]
    inputs['labels'] = torch.Tensor(samples['labels'])

    return inputs

In [ ]:
desc_transformed_data = dataset.map(desc_batch_transform, 
                                    batched=True, 
                                    remove_columns=dataset['train'].column_names)
desc_transformed_data

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 5119
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 1280
    })
})

In [ ]:
desc_transformed_data.set_format('torch')

In [ ]:
desc_transformed_data['train'].features
new_feat = desc_transformed_data['train'].features.copy()
new_feat['labels'] = Sequence(feature=Value('float32'))
desc_transformed_data['train'] = desc_transformed_data['train'].cast(new_feat)
desc_transformed_data['test'] = desc_transformed_data['test'].cast(new_feat)
# desc_transformed_data['validation'] = desc_transformed_data['validation'].cast(new_feat)

Casting the dataset:   0%|          | 0/6 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/2 [00:00<?, ?ba/s]

In [ ]:
desc_transformed_data['train'][0]

{'labels': tensor([0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.]),
 'input_ids': tensor([    0,   104,  5971,   821,  8299,     6,  7434,    29,     8,   786,
            12, 33430,   405,  4668, 32937,    25,   741, 16250,  9236,     4,
         11185,     4,  8713,     4,  3848, 19549,  9413,     8,    39,  4025,
             9,   522, 19200,  6136,  1337, 37578,  1200,     4,     2,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,     1,
             1,     1,     1,     1,     1,     1,     1,     1,     1,

In [ ]:
def multilabel_metrics(predictions, labels, threshold=0.5):
    # apply sigmoid on predictions
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # compute metrics
    y_true = labels
    f1_micro_avg = f1_score(y_true, y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average='micro')
    accuracy = accuracy_score(y_true, y_pred)

    metrics = {'f1': f1_micro_avg,
                'roc_auc': roc_auc,
                'accuracy': accuracy}

    return metrics

def compute_metrics(p):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    result = multilabel_metrics(predictions=preds, labels=p.label_ids)
    return result

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

#### Fine Tune

In [ ]:
desc_model = AutoModelForSequenceClassification.from_pretrained(bart_ckpt, num_labels=len(labels),
                                           id2label=id2label, label2id=label2id,
                                           problem_type='multi_label_classification')

desc_model = desc_model.to(device)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--facebook--bart-base/snapshots/aadd2ab0ae0c8268c7c9693540e9904811f36177/config.json
Model config BartConfig {
  "_name_or_path": "facebook/bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "Action",
    "1": "Adventure",
    "2"

In [ ]:
batch_size = 64
logging_steps = len(desc_transformed_data['train']) // batch_size

In [ ]:
training_args = TrainingArguments(output_dir='/content/',
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  evaluation_strategy='epoch',
                                  save_strategy='epoch',
                                  num_train_epochs=10,
                                  fp16=True if torch.cuda.is_available() else False,
                                  logging_steps=logging_steps,
                                  learning_rate=5e-5,
                                  save_total_limit=2,
                                  load_best_model_at_end=True)

In [ ]:
desc_trainer = Trainer(model=desc_model,
                  args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=desc_transformed_data['train'],
                  eval_dataset=desc_transformed_data['test'],
                  tokenizer=tokenizer)

desc_train_result = desc_trainer.train()

Using cuda_amp half precision backend
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 5119
  Num Epochs = 10
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 800
  Number of trainable parameters = 140029464


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.281000,0.228258,0.469533,0.670380,0.082031
2,0.208100,0.195445,0.565757,0.722101,0.118750
3,0.178900,0.187573,0.583802,0.733500,0.147656
4,0.161700,0.183778,0.603746,0.750366,0.165625
5,0.147400,0.184080,0.607829,0.754731,0.158594
6,0.137900,0.182902,0.615194,0.759780,0.169531
7,0.127800,0.183973,0.621138,0.764026,0.182812
8,0.120000,0.186471,0.622382,0.769365,0.183594
9,0.114100,0.187505,0.624448,0.772753,0.189062
10,0.111700,0.186836,0.624248,0.769915,0.191406


***** Running Evaluation *****
  Num examples = 1280
  Batch size = 64
Saving model checkpoint to /content/checkpoint-80
Configuration saved in /content/checkpoint-80/config.json
Model weights saved in /content/checkpoint-80/pytorch_model.bin
tokenizer config file saved in /content/checkpoint-80/tokenizer_config.json
Special tokens file saved in /content/checkpoint-80/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 1280
  Batch size = 64
Saving model checkpoint to /content/checkpoint-160
Configuration saved in /content/checkpoint-160/config.json
Model weights saved in /content/checkpoint-160/pytorch_model.bin
tokenizer config file saved in /content/checkpoint-160/tokenizer_config.json
Special tokens file saved in /content/checkpoint-160/special_tokens_map.json
Deleting older checkpoint [/content/checkpoint-85] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 1280
  Batch size = 64
Saving model checkpoint to /content/checkpoint-240
Con

In [ ]:
desc_trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1280
  Batch size = 64


{'eval_loss': 0.18290236592292786,
 'eval_f1': 0.6151943462897527,
 'eval_roc_auc': 0.7597797457449308,
 'eval_accuracy': 0.16953125,
 'eval_runtime': 3.7377,
 'eval_samples_per_second': 342.46,
 'eval_steps_per_second': 5.351,
 'epoch': 10.0}

In [ ]:
# desc_trainer.predict(desc_transformed_data['test']).metrics

In [ ]:
desc_trainer.save_model('/content/drive/MyDrive/ITMO Master/datasets/ml-industry/models/desc-bart-pretrained/')

Saving model checkpoint to /content/drive/MyDrive/ITMO Master/datasets/ml-industry/models/desc-bart-pretrained/
Configuration saved in /content/drive/MyDrive/ITMO Master/datasets/ml-industry/models/desc-bart-pretrained/config.json
Model weights saved in /content/drive/MyDrive/ITMO Master/datasets/ml-industry/models/desc-bart-pretrained/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/ITMO Master/datasets/ml-industry/models/desc-bart-pretrained/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/ITMO Master/datasets/ml-industry/models/desc-bart-pretrained/special_tokens_map.json


In [ ]:
torch.cuda.empty_cache()

#### Extract Logits

In [ ]:
desc_model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/ITMO Master/datasets/ml-industry/models/desc-bart-pretrained/')
desc_model = desc_model.to(device)

loading configuration file /content/drive/MyDrive/ITMO Master/datasets/ml-industry/models/desc-bart-pretrained/config.json
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'Action', '1': 'Adventure', '2': 'Animation', '3': 'Biography', '4': 'Comedy', '5': 'Crime', '6': 'Documentary', '7': 'Drama', '8': 'Family', '9': 'Fantasy', '10': 'Film Noir', '11': 'History', '12': 'Horror', '13': 'Music', '14': 'Musical', '15': 'Mystery', '16': 'Romance', '17': 'Sci-Fi', '18': 'Short Film', '19': 'Sport', '20': 'Superhero', '21': 'Thriller', '22': 'War', '23': 'Western'}. The number of labels wil be overwritten to 24.
Model config BartConfig {
  "_name_or_path": "/content/drive/MyDrive/ITMO Master/datasets/ml-industry/models/desc-bart-pretrained/",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "bos_token_

In [ ]:
# create loader
trainloader = DataLoader(desc_transformed_data['train'], batch_size=batch_size)
# validloader = DataLoader(desc_transformed_data['validation'], batch_size=batch_size)
testloader = DataLoader(desc_transformed_data['test'], batch_size=batch_size)

In [ ]:
# collect all train logits
train_logits = []

for batch in tqdm(trainloader):
    batch = {k: v.to(device) for k, v in batch.items()}
    out = desc_model(**batch).logits.detach().cpu()
    train_logits.append(out)

# save it as pickle
with open('/content/drive/MyDrive/ITMO Master/datasets/ml-industry/models/desc-bart-logits/train-logits.pkl', 'wb') as f:
    logits = torch.cat(train_logits)
    pickle.dump(logits, f)

# free some gpu memory
torch.cuda.empty_cache()

  0%|          | 0/80 [00:00<?, ?it/s]

In [ ]:
# # collect all valid logits
# valid_logits = []

# for batch in tqdm(validloader):
#     batch = {k: v.to(device) for k, v in batch.items()}
#     out = desc_model(**batch).logits.detach().cpu()
#     valid_logits.append(out)

# # save it as pickle
# with open('/content/drive/MyDrive/ITMO Master/datasets/ml-industry/models/desc-bart-logits/valid-logits.pkl', 'wb') as f:
#     logits = torch.cat(valid_logits)
#     pickle.dump(logits, f)

# # free some gpu memory
# torch.cuda.empty_cache()

In [ ]:
# collect all test logits
test_logits = []

for batch in tqdm(testloader):
    batch = {k: v.to(device) for k, v in batch.items()}
    out = desc_model(**batch).logits.detach().cpu()
    test_logits.append(out)

# save it as pickle
with open('/content/drive/MyDrive/ITMO Master/datasets/ml-industry/models/desc-bart-logits/test-logits.pkl', 'wb') as f:
    logits = torch.cat(test_logits)
    pickle.dump(logits, f)

# free some gpu memory
torch.cuda.empty_cache()

  0%|          | 0/20 [00:00<?, ?it/s]

### Title Model

In [ ]:
def title_batch_transform(samples):
    # print(samples)
    # tokenize
    inputs = tokenizer([i for i in samples['title']],
                       max_length=128,
                       padding='max_length',
                       truncation=True,
                       return_attention_mask=True,
                       return_tensors='pt')
    # add labels
    # inputs['labels'] = [torch.Tensor(i).float() for i in samples['labels']]
    inputs['labels'] = torch.Tensor(samples['labels'])

    return inputs

In [ ]:
title_transformed_data = dataset.map(title_batch_transform, 
                                    batched=True, 
                                    remove_columns=dataset['train'].column_names)
title_transformed_data

  0%|          | 0/6 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 5119
    })
    test: Dataset({
        features: ['labels', 'input_ids', 'attention_mask'],
        num_rows: 1280
    })
})

In [ ]:
title_transformed_data.set_format('torch')

In [ ]:
title_transformed_data['train'].features
new_feat = title_transformed_data['train'].features.copy()
new_feat['labels'] = Sequence(feature=Value('float32'))
title_transformed_data['train'] = title_transformed_data['train'].cast(new_feat)
title_transformed_data['test'] = title_transformed_data['test'].cast(new_feat)
# title_transformed_data['validation'] = title_transformed_data['validation'].cast(new_feat)

Casting the dataset:   0%|          | 0/6 [00:00<?, ?ba/s]

Casting the dataset:   0%|          | 0/2 [00:00<?, ?ba/s]

#### Fine Tune

In [ ]:
batch_size = 64
logging_steps = len(title_transformed_data['train']) // batch_size

In [ ]:
training_args = TrainingArguments(output_dir='/content/',
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  evaluation_strategy='epoch',
                                  save_strategy='epoch',
                                  num_train_epochs=10,
                                  fp16=True if torch.cuda.is_available() else False,
                                  logging_steps=logging_steps,
                                  learning_rate=5e-5,
                                  save_total_limit=2,
                                  load_best_model_at_end=True)

In [ ]:
title_model = AutoModelForSequenceClassification.from_pretrained(bart_ckpt, num_labels=len(labels),
                                           id2label=id2label, label2id=label2id,
                                           problem_type='multi_label_classification')

title_model = title_model.to(device)

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--facebook--bart-base/snapshots/aadd2ab0ae0c8268c7c9693540e9904811f36177/config.json
Model config BartConfig {
  "_name_or_path": "facebook/bart-base",
  "activation_dropout": 0.1,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": false,
  "architectures": [
    "BartModel"
  ],
  "attention_dropout": 0.1,
  "bos_token_id": 0,
  "classif_dropout": 0.1,
  "classifier_dropout": 0.0,
  "d_model": 768,
  "decoder_attention_heads": 12,
  "decoder_ffn_dim": 3072,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 6,
  "decoder_start_token_id": 2,
  "dropout": 0.1,
  "early_stopping": true,
  "encoder_attention_heads": 12,
  "encoder_ffn_dim": 3072,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 6,
  "eos_token_id": 2,
  "forced_bos_token_id": 0,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "Action",
    "1": "Adventure",
    "2"

In [ ]:
title_trainer = Trainer(model=title_model,
                  args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=title_transformed_data['train'],
                  eval_dataset=title_transformed_data['test'],
                  tokenizer=tokenizer)

title_train_result = title_trainer.train()

Using cuda_amp half precision backend
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 5119
  Num Epochs = 10
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 800
  Number of trainable parameters = 140029464


Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.301800,0.279797,0.001254,0.500296,0.000000
2,0.276800,0.267240,0.279489,0.583030,0.032813
3,0.261500,0.255848,0.352106,0.613144,0.056250
4,0.243500,0.250538,0.383485,0.629109,0.069531
5,0.226900,0.246933,0.398268,0.634993,0.065625
6,0.212400,0.244517,0.432046,0.654010,0.079687
7,0.200700,0.249663,0.421138,0.649844,0.088281
8,0.191800,0.251265,0.419198,0.649343,0.087500
9,0.185200,0.250626,0.447935,0.666365,0.099219
10,0.180800,0.250681,0.439616,0.661637,0.095312


***** Running Evaluation *****
  Num examples = 1280
  Batch size = 64
Saving model checkpoint to /content/checkpoint-80
Configuration saved in /content/checkpoint-80/config.json
Model weights saved in /content/checkpoint-80/pytorch_model.bin
tokenizer config file saved in /content/checkpoint-80/tokenizer_config.json
Special tokens file saved in /content/checkpoint-80/special_tokens_map.json
Deleting older checkpoint [/content/checkpoint-480] due to args.save_total_limit
***** Running Evaluation *****
  Num examples = 1280
  Batch size = 64
Saving model checkpoint to /content/checkpoint-160
Configuration saved in /content/checkpoint-160/config.json
Model weights saved in /content/checkpoint-160/pytorch_model.bin
tokenizer config file saved in /content/checkpoint-160/tokenizer_config.json
Special tokens file saved in /content/checkpoint-160/special_tokens_map.json
Deleting older checkpoint [/content/checkpoint-800] due to args.save_total_limit
***** Running Evaluation *****
  Num exampl

In [ ]:
title_trainer.evaluate()

***** Running Evaluation *****
  Num examples = 1280
  Batch size = 64


{'eval_loss': 0.24451717734336853,
 'eval_f1': 0.43204577968526464,
 'eval_roc_auc': 0.6540097928250843,
 'eval_accuracy': 0.0796875,
 'eval_runtime': 3.793,
 'eval_samples_per_second': 337.468,
 'eval_steps_per_second': 5.273,
 'epoch': 10.0}

In [ ]:
# title_trainer.predict(title_transformed_data['test']).metrics

In [ ]:
title_trainer.save_model('/content/drive/MyDrive/ITMO Master/datasets/ml-industry/models/title-bart-pretrained/')

Saving model checkpoint to /content/drive/MyDrive/ITMO Master/datasets/ml-industry/models/title-bart-pretrained/
Configuration saved in /content/drive/MyDrive/ITMO Master/datasets/ml-industry/models/title-bart-pretrained/config.json
Model weights saved in /content/drive/MyDrive/ITMO Master/datasets/ml-industry/models/title-bart-pretrained/pytorch_model.bin
tokenizer config file saved in /content/drive/MyDrive/ITMO Master/datasets/ml-industry/models/title-bart-pretrained/tokenizer_config.json
Special tokens file saved in /content/drive/MyDrive/ITMO Master/datasets/ml-industry/models/title-bart-pretrained/special_tokens_map.json


In [ ]:
torch.cuda.empty_cache()

#### Extract Logits

In [ ]:
title_model = AutoModelForSequenceClassification.from_pretrained('/content/drive/MyDrive/ITMO Master/datasets/ml-industry/models/title-bart-pretrained/')
title_model = title_model.to(device)

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'Action', '1': 'Adventure', '2': 'Animation', '3': 'Biography', '4': 'Comedy', '5': 'Crime', '6': 'Documentary', '7': 'Drama', '8': 'Family', '9': 'Fantasy', '10': 'Film Noir', '11': 'History', '12': 'Horror', '13': 'Music', '14': 'Musical', '15': 'Mystery', '16': 'Romance', '17': 'Sci-Fi', '18': 'Short Film', '19': 'Sport', '20': 'Superhero', '21': 'Thriller', '22': 'War', '23': 'Western'}. The number of labels wil be overwritten to 24.


In [ ]:
# create loader
trainloader = DataLoader(title_transformed_data['train'], batch_size=batch_size)
# validloader = DataLoader(title_transformed_data['validation'], batch_size=batch_size)
testloader = DataLoader(title_transformed_data['test'], batch_size=batch_size)

In [ ]:
# collect all train logits
train_logits = []

for batch in tqdm(trainloader):
    batch = {k: v.to(device) for k, v in batch.items()}
    out = title_model(**batch).logits.detach().cpu()
    train_logits.append(out)

# save it as pickle
with open('/content/drive/MyDrive/ITMO Master/datasets/ml-industry/models/title-bart-logits/train-logits.pkl', 'wb') as f:
    logits = torch.cat(train_logits)
    pickle.dump(logits, f)

# free some gpu memory
torch.cuda.empty_cache()

  0%|          | 0/80 [00:00<?, ?it/s]

In [ ]:
# # collect all valid logits
# valid_logits = []

# for batch in tqdm(validloader):
#     batch = {k: v.to(device) for k, v in batch.items()}
#     out = title_model(**batch).logits.detach().cpu()
#     valid_logits.append(out)

# # save it as pickle
# with open('/content/drive/MyDrive/ITMO Master/datasets/ml-industry/models/title-bart-logits/valid-logits.pkl', 'wb') as f:
#     logits = torch.cat(valid_logits)
#     pickle.dump(logits, f)

# # free some gpu memory
# torch.cuda.empty_cache()

In [ ]:
# collect all test logits
test_logits = []

for batch in tqdm(testloader):
    batch = {k: v.to(device) for k, v in batch.items()}
    out = title_model(**batch).logits.detach().cpu()
    test_logits.append(out)

# save it as pickle
with open('/content/drive/MyDrive/ITMO Master/datasets/ml-industry/models/title-bart-logits/test-logits.pkl', 'wb') as f:
    logits = torch.cat(test_logits)
    pickle.dump(logits, f)

# free some gpu memory
torch.cuda.empty_cache()

  0%|          | 0/20 [00:00<?, ?it/s]

In [ ]:
# collect all labels
with open('/content/drive/MyDrive/ITMO Master/datasets/ml-industry/models/train-labels.pkl', 'wb') as f:
    labels = desc_transformed_data['train']['labels']
    pickle.dump(labels, f)

# with open('/content/drive/MyDrive/ITMO Master/datasets/ml-industry/models/valid-labels.pkl', 'wb') as f:
#     labels = desc_transformed_data['validation']['labels']
#     pickle.dump(labels, f)

with open('/content/drive/MyDrive/ITMO Master/datasets/ml-industry/models/test-labels.pkl', 'wb') as f:
    labels = desc_transformed_data['test']['labels']
    pickle.dump(labels, f)